## A Recommender System for Groceries Contractor

In [1]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries are imported.')

Libraries are imported.


### Postal Codes in Toronto

In [2]:
# Loading the dataset which is about postal codes in Toronto
# This dataset was created in week 2
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

@hidden_cell
# he following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
#You might want to remove those credentials before you share your notebook.
client_8929f92caa6a4693939627aebd795589 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eGqfCno4t1-abgP7_NMbTBGYewG3pglgkVaV36p0QWXY',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_8929f92caa6a4693939627aebd795589.get_object(Bucket='neighborhoodtorotoclustering-donotdelete-pr-dsxwwoetyfclx1',Key='toronto.csv')['Body']
#add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_toronto = pd.read_csv(body)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,Mississauga,Parkwoods,43.753259,-79.329656
1,M4A,Mississauga,Victoria Village,43.725882,-79.315572
2,M5A,Mississauga,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,Mississauga,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Mississauga,Queen's Park,43.662301,-79.389494


### Map of Toronto City (with its Postal Codes' regions)

In [3]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### Focusing on the "Mississauga" Borough in Toronto (its neighborhoods)

In [4]:
mississauga_data = df_toronto[df_toronto['Borough'] == 'Mississauga'].reset_index(drop=True)
mississauga_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,Mississauga,Parkwoods,43.753259,-79.329656
1,M4A,Mississauga,Victoria Village,43.725882,-79.315572
2,M5A,Mississauga,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,Mississauga,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Mississauga,Queen's Park,43.662301,-79.389494


### Create a Map of Mississauga and Its Neighbourhoods

In [5]:
_address = 'Mississauga,Toronto'
_latitude = 43.595310
_longitude = -79.640579
print('Geograpical coordinate of Mississauga: {}, {}.'.format(_latitude, _longitude))

Geograpical coordinate of Mississauga: 43.59531, -79.640579.


In [6]:
map_missi = folium.Map(location=[_latitude, _longitude], zoom_start=12)

#add markers to map
for lat, lng, label in zip(mississauga_data['Latitude'], mississauga_data['Longitude'], mississauga_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_missi)  
    
map_missi

In [7]:
def getNearbyVenues (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        #counter += 1
        #print('{}.'.format(counter))
        #print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;


In [8]:
# @hiddel_cell
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' # your Foursquare ID
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Crawling Internet (in fact only Foursquare database) for 
### Venues in the Neighborhoods inside "Mississauga"


In [9]:
print('Crawling neighborhoods inside "Mississauga"')
Mississauga_foursquare_dataset = getNearbyVenues(list(mississauga_data['Postal Code']),
                                                   list(mississauga_data['Neighbourhood']),
                                                   list(mississauga_data['Latitude']),
                                                   list(mississauga_data['Longitude']),)

Crawling neighborhoods inside "Mississauga"


# Breakpoint:
## Saving results of Foursquare, so that we would not need to connect every time to Foursquare (and use our portions) .

In [10]:
import pickle
with open("Missisauga_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Mississauga_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')  

Received Data from Internet is Saved to Computer.


In [11]:
with open("Missisauga_foursquare_dataset.txt", "rb") as fp:   # Unpickling
   Mississauga_foursquare_dataset = pickle.load(fp)

### Cleaning the RAW Data Received from Foursquare Database

In [12]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [13]:
mississauga_venues = get_venue_dataset(Mississauga_foursquare_dataset)

Number of Venuse in Coordination "M3A" Posal Code and "Parkwoods" Negihborhood(s) is:
28
Number of Venuse in Coordination "M4A" Posal Code and "Victoria Village" Negihborhood(s) is:
11
Number of Venuse in Coordination "M5A" Posal Code and "Harbourfront,Regent Park" Negihborhood(s) is:
100
Number of Venuse in Coordination "M6A" Posal Code and "Lawrence Heights,Lawrence Manor" Negihborhood(s) is:
56
Number of Venuse in Coordination "M7A" Posal Code and "Queen's Park" Negihborhood(s) is:
100
Number of Venuse in Coordination "M9A" Posal Code and "Islington Avenue" Negihborhood(s) is:
14
Number of Venuse in Coordination "M1B" Posal Code and "Rouge,Malvern" Negihborhood(s) is:
18
Number of Venuse in Coordination "M3B" Posal Code and "Don Mills North" Negihborhood(s) is:
29
Number of Venuse in Coordination "M4B" Posal Code and "Woodbine Gardens,Parkview Hill" Negihborhood(s) is:
20
Number of Venuse in Coordination "M5B" Posal Code and "Ryerson,Garden District" Negihborhood(s) is:
100
Number o

### Showing Venues for Each Neighborhood in Mississauga

In [14]:
mississauga_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M3A,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,This spot is popular,Caribbean Restaurant,833
1,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,This spot is popular,Park,245
2,M3A,Parkwoods,43.753259,-79.329656,Tim Hortons,This spot is popular,Café,866
3,M3A,Parkwoods,43.753259,-79.329656,A&W Canada,This spot is popular,Fast Food Restaurant,852
4,M3A,Parkwoods,43.753259,-79.329656,Food Basics,This spot is popular,Supermarket,895


In [15]:
mississauga_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
4883,M8Z,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,Fresh & Tasty,This spot is popular,Deli / Bodega,786
4884,M8Z,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,Mr.Sub,This spot is popular,Restaurant,786
4885,M8Z,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,High Seas Restaurant,This spot is popular,Mediterranean Restaurant,804
4886,M8Z,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,Pet Valu,This spot is popular,Pet Store,932
4887,M8Z,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999,Sleep Country,This spot is popular,Mattress Store,953


# Breakpoint: 
## End of Processing the Retrieved Information from Foursquare
## Saving a Cleaned Version of DataFrame as the Results from Foursquare 

In [16]:
mississauga_venues.to_csv('mississauga_venues.csv')

### Loading Data from File (Saved "Foursquare " DataFrame for Venues)

In [17]:
mississauga_venues = pd.read_csv('mississauga_venues.csv')

### Some Summary Information about Neighborhoods inside "Mississauga"

In [18]:
nb_list = list(mississauga_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Mississauga:')
print(len(nb_list))
print('List of Neighborhoods inside Mississauga:')
nb_list

Number of Neighborhoods inside Mississauga:
102
List of Neighborhoods inside Mississauga:


['Parkwoods',
 'Victoria Village',
 'Harbourfront,Regent Park',
 'Lawrence Heights,Lawrence Manor',
 "Queen's Park",
 'Islington Avenue',
 'Rouge,Malvern',
 'Don Mills North',
 'Woodbine Gardens,Parkview Hill',
 'Ryerson,Garden District',
 'Glencairn',
 'Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park',
 'Highland Creek,Rouge Hill,Port Union',
 'Flemingdon Park,Don Mills South',
 'Woodbine Heights',
 'St. James Town',
 'Humewood-Cedarvale',
 'Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe',
 'Guildwood,Morningside,West Hill',
 'The Beaches',
 'Berczy Park',
 'Caledonia-Fairbanks',
 'Woburn',
 'Leaside',
 'Central Bay Street',
 'Christie',
 'Cedarbrae',
 'Hillcrest Village',
 'Bathurst Manor,Downsview North,Wilson Heights',
 'Thorncliffe Park',
 'Adelaide,King,Richmond',
 'Dovercourt Village,Dufferin',
 'Scarborough Village',
 'Fairview,Henry Farm,Oriole',
 'Northwood Park,York University',
 'East Toronto',
 'Harbourfront East,Toronto Islands,Union Station

### Some Summary Information about Neighborhoods inside "Mississauga" Cont'd

In [19]:
nb_venue_summary = mississauga_venues.groupby('Neighborhood').count()
nb_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100,100
Agincourt,45,45,45,45,45,45,45
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",24,24,24,24,24,24,24
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",18,18,18,18,18,18,18
"Alderwood,Long Branch",28,28,28,28,28,28,28


In [20]:
print('There are {} uniques categories.'.format(len(mississauga_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(mississauga_venues.loc[mississauga_venues['Venue Category'] != 'Neighborhood']['Venue Category'].sort_values().unique())

There are 327 uniques categories.
Here is the list of different categories:


['Accessories Store',
 'Adult Boutique',
 'Afghan Restaurant',
 'African Restaurant',
 'Airport',
 'Airport Lounge',
 'American Restaurant',
 'Amphitheater',
 'Animal Shelter',
 'Antique Shop',
 'Aquarium',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Dealership',
 'Auto Garage',
 'Auto Workshop',
 'Automotive Shop',
 'BBQ Joint',
 'Baby Store',
 'Badminton Court',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Stadium',
 'Beach',
 'Beach Bar',
 'Beer Bar',
 'Beer Store',
 'Belgian Restaurant',
 'Bike Shop',
 'Bistro',
 'Bookstore',
 'Boutique',
 'Bowling Alley',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bridal Shop',
 'Bridge',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Bus Station',
 'Bus Stop',
 'Business Service',
 'Butcher',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Cantonese Restaurant',
 '

In [21]:
mississauga_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 9 columns):
Unnamed: 0                4888 non-null int64
Postal Code               4888 non-null object
Neighborhood              4888 non-null object
Neighborhood Latitude     4888 non-null float64
Neighborhood Longitude    4888 non-null float64
Venue                     4888 non-null object
Venue Summary             4888 non-null object
Venue Category            4888 non-null object
Distance                  4888 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 343.8+ KB


In [22]:
# Just for fun and deeper understanding
print(type(mississauga_venues[['Venue Category']]))

print(type(mississauga_venues['Venue Category']))

mississauga_venues = mississauga_venues.loc[mississauga_venues['Venue Category'] != 'Neighborhood']


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


### One-hot Encoding the "categroies" Column into Every Unique Categorical Feature.

In [23]:
# one hot encoding
mississauga_onehot = pd.get_dummies(data = mississauga_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
mississauga_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Par

In [24]:
mississauga_onehot['Neighborhood'].unique()

array(['Parkwoods', 'Victoria Village', 'Harbourfront,Regent Park',
       'Lawrence Heights,Lawrence Manor', "Queen's Park",
       'Islington Avenue', 'Rouge,Malvern', 'Don Mills North',
       'Woodbine Gardens,Parkview Hill', 'Ryerson,Garden District',
       'Glencairn',
       'Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park',
       'Highland Creek,Rouge Hill,Port Union',
       'Flemingdon Park,Don Mills South', 'Woodbine Heights',
       'St. James Town', 'Humewood-Cedarvale',
       'Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe',
       'Guildwood,Morningside,West Hill', 'The Beaches', 'Berczy Park',
       'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street',
       'Christie', 'Cedarbrae', 'Hillcrest Village',
       'Bathurst Manor,Downsview North,Wilson Heights', 'Thorncliffe Park',
       'Adelaide,King,Richmond', 'Dovercourt Village,Dufferin',
       'Scarborough Village', 'Fairview,Henry Farm,Oriole',
       'Northwood Park,

## Manually Selecting (Subsetting) Related Features for the Groceries Contractor

In [25]:
# This list is created manually 
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant',

 'BBQ Joint',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 
 'Diner',

 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Fruit & Vegetable Store',
 
 'Greek Restaurant',
 'Grocery Store',
 
 'Hakka Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
    
 'Japanese Restaurant',
    
 'Korean Restaurant',
    
 'Latin American Restaurant',

 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 
 'Noodle House',
 
 'Pizza Place',
 
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shanghai Restaurant',
 'Sushi Restaurant',
    
 'Taiwanese Restaurant',
 'Thai Restaurant',
 
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
    
 'Wings Joint']

### Updating the One-hot Encoded DataFrame and grouping the data by Neighborhoods

In [26]:
mississauga_onehot = mississauga_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


mississauga_onehot.head()

,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Greek Restaurant,Grocery Store,Hakka Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Adelaide,King,Richmond",0,4,2,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,3,1,0,1,0,3,0,1,2,0,0
Agincourt,0,1,1,0,2,1,0,0,1,2,6,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,2,1,2,1,1,1,0,0,0,1,0
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",0,0,0,1,2,0,0,0,0,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,2,0,0,0,0,0,0,0,1,0,0
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0
"Alderwood,Long Branch",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0


## Integrating Different Restaurants

In [27]:
feat_name_list = list(mississauga_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
mississauga_onehot['Total Restaurants'] = mississauga_onehot[restaurant_list].sum(axis = 1)
mississauga_onehot = mississauga_onehot.drop(columns = restaurant_list)


feat_name_list = list(mississauga_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
mississauga_onehot['Total Joints'] = mississauga_onehot[joint_list].sum(axis = 1)
mississauga_onehot = mississauga_onehot.drop(columns = joint_list)

### Processed DataFrame about neighborhoods inside Mississauga.

In [28]:
mississauga_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,
"Adelaide,King,Richmond",2,1,0,0,0,0,0,1,3,0,18,1
Agincourt,2,1,0,0,0,0,1,1,2,2,19,0
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,0,0,0,0,0,0,1,2,0,9,1
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",0,0,0,0,0,0,3,0,3,1,1,1
"Alderwood,Long Branch",0,0,0,0,0,0,2,0,2,1,0,0
"Bathurst Manor,Downsview North,Wilson Heights",0,0,1,0,0,0,0,0,2,1,5,1
Bayview Village,0,0,0,0,0,0,1,0,0,0,4,0
"Bedford Park,Lawrence Manor East",1,0,0,0,0,0,1,0,1,1,13,1
Berczy Park,3,3,1,1,0,0,1,0,1,1,14,2


# Run k-means to Cluster Neighborhoods into 5 Clusters

In [29]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(mississauga_onehot)


## Showing Centers of Each Cluster

In [30]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = mississauga_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G1,1.666667,0.666667,1.555556,1.111111e-01,2.222222e-01,1.111111e-01,1.666667,0.000000e+00,2.777778,1.888889,27.111111,1.555556,39.333333
G4,2.500000,1.375000,0.687500,1.250000e-01,2.775558e-17,6.938894e-18,0.687500,3.750000e-01,2.000000,1.000000,19.750000,1.500000,30.000000
G3,1.722222,0.777778,0.388889,1.666667e-01,1.111111e-01,6.938894e-18,1.222222,5.555556e-02,1.611111,1.333333,13.166667,1.888889,22.444444
G5,0.809524,0.380952,0.333333,4.761905e-02,9.523810e-02,4.761905e-02,0.761905,4.761905e-02,1.380952,0.714286,7.523810,1.285714,13.428571
G2,0.289474,0.157895,0.131579,-2.775558e-17,5.263158e-02,-6.938894e-18,0.578947,-2.775558e-17,1.131579,0.447368,1.789474,0.289474,4.868421


### Result:
#### Best group is G3;
#### Second group is G2;
#### Third group is G4;

### Inserting "kmeans.labels_" into the original Mississauga DataFrame
#### Finding the corresponding group for each Neighborhood.

In [31]:
nb_summary = pd.DataFrame([mississauga_onehot.index, 1 + kmeans.labels_]).T
nb_summary.columns = ['Neighborhood', 'Group']
nb_summary

,Neighborhood,Group
0,"Adelaide,King,Richmond",4
1,Agincourt,4
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",5
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",2
4,"Alderwood,Long Branch",2
5,"Bathurst Manor,Downsview North,Wilson Heights",5
6,Bayview Village,2
7,"Bedford Park,Lawrence Manor East",3
8,Berczy Park,3
9,"Birch Cliff,Cliffside West",2


# Results:
## Best Neighborhood:

In [32]:
nb_summary[nb_summary['Group'] == 3]

,Neighborhood,Group
7,"Bedford Park,Lawrence Manor East",3
8,Berczy Park,3
11,"Brockton,Exhibition Place,Parkdale Village",3
24,"Clarks Corners,Sullivan,Tam O'Shanter",3
30,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",3
34,"Dorset Park,Scarborough Town Centre,Wexford He...",3
35,"Dovercourt Village,Dufferin",3
45,"Flemingdon Park,Don Mills South",3
46,"Forest Hill North,Forest Hill West",3
47,Glencairn,3


In [33]:
name_of_nb = list(nb_summary[nb_summary['Group'] == 3]['Neighborhood'])
name_of_nb

['Bedford Park,Lawrence Manor East',
 'Berczy Park',
 'Brockton,Exhibition Place,Parkdale Village',
 "Clarks Corners,Sullivan,Tam O'Shanter",
 'Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West',
 'Dorset Park,Scarborough Town Centre,Wexford Heights',
 'Dovercourt Village,Dufferin',
 'Flemingdon Park,Don Mills South',
 'Forest Hill North,Forest Hill West',
 'Glencairn',
 'Harbourfront East,Toronto Islands,Union Station',
 'Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor',
 'Lawrence Heights,Lawrence Manor',
 'Moore Park,Summerhill East',
 'Parkdale,Roncesvalles',
 'Runnymede,Swansea',
 'The Beaches',
 'The Beaches West,India Bazaar']

## Second Neighborhoods

In [34]:
nb_summary[nb_summary['Group'] == 2]

,Neighborhood,Group
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",2
4,"Alderwood,Long Branch",2
6,Bayview Village,2
9,"Birch Cliff,Cliffside West",2
10,"Bloordale Gardens,Eringate,Markland Wood,Old B...",2
13,"CFB Toronto,Downsview East",2
14,"CN Tower,Bathurst Quay,Island airport,Harbourf...",2
16,Caledonia-Fairbanks,2
23,"Clairlea,Golden Mile,Oakridge",2
25,"Cliffcrest,Cliffside,Scarborough Village West",2


In [35]:
name_of_nb = list(nb_summary[nb_summary['Group'] == 2]['Neighborhood'])
name_of_nb

['Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown',
 'Alderwood,Long Branch',
 'Bayview Village',
 'Birch Cliff,Cliffside West',
 'Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe',
 'CFB Toronto,Downsview East',
 'CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara',
 'Caledonia-Fairbanks',
 'Clairlea,Golden Mile,Oakridge',
 'Cliffcrest,Cliffside,Scarborough Village West',
 'Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park',
 'Del Ray,Keelesdale,Mount Dennis,Silverthorn',
 'Downsview Central',
 'Downsview West',
 'Downsview,North Park,Upwood Park',
 'Emery,Humberlea',
 'Guildwood,Morningside,West Hill',
 'Highland Creek,Rouge Hill,Port Union',
 'Hillcrest Village',
 "Humber Bay,King's Mill Park,Kingsway Park South East,Mimico NE,Old Mill South,The Queensway East,Royal York South East,Sunnylea",
 'Humber Summit',
 'Humewood-Cedarvale',
 'Islington Avenue',

## Third Neighborhood

In [36]:
nb_summary[nb_summary['Group'] == 4]

,Neighborhood,Group
0,"Adelaide,King,Richmond",4
1,Agincourt,4
17,Canada Post Gateway Processing Centre,4
19,Central Bay Street,4
20,"Chinatown,Grange Park,Kensington Market",4
27,"Commerce Court,Victoria Hotel",4
32,"Design Exchange,Toronto Dominion Centre",4
41,East Toronto,4
44,"First Canadian Place,Underground city",4
49,"Harbord,University of Toronto",4


In [37]:
name_of_nb = list(nb_summary[nb_summary['Group'] == 4]['Neighborhood'])
name_of_nb

['Adelaide,King,Richmond',
 'Agincourt',
 'Canada Post Gateway Processing Centre',
 'Central Bay Street',
 'Chinatown,Grange Park,Kensington Market',
 'Commerce Court,Victoria Hotel',
 'Design Exchange,Toronto Dominion Centre',
 'East Toronto',
 'First Canadian Place,Underground city',
 'Harbord,University of Toronto',
 'Harbourfront,Regent Park',
 'High Park,The Junction South',
 'Little Portugal,Trinity',
 "Queen's Park",
 'St. James Town',
 'Stn A PO Boxes 25 The Esplanade']

### Thank you ;)
